# Overview

In notebook [Loading data for gnndelete](https://www.kaggle.com/code/aisuko/loading-data-for-gnndelete), we failed to manipulate data. However, we deal with them by using RMIT race hub aws service. Currently, we release these data on:

* https://www.kaggle.com/models/skywardai/ogbl-collab
* https://www.kaggle.com/models/skywardai/ogbl-citation2

I released them as models due to easier for us to do testing on Kaggle. They aren't models. Please noticed.

An also, we will twisted the code from GNNdelete project https://github.com/mims-harvard/GNNDelete.

# What we do here?

In this notebook, we will learn and save all the parameters to json file. And we can load all these parameters in the other notebooks.

In [ ]:
import argparse

# Acknowledgement

* https://github.com/SkywardAI/GNNDelete/blob/main/framework/training_args.py